In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('/content/bank-full.csv', delimiter=';')

features = ['age', 'job', 'marital', 'education', 'balance', 'housing', 'loan',
            'contact', 'duration', 'campaign', 'poutcome']
X = df[features]
y = df['y']

X['housing'] = X['housing'].map({'yes': 1, 'no': 0})
X['loan'] = X['loan'].map({'yes': 1, 'no': 0})


y = y.map({'yes': 1, 'no': 0})

X_encoded = pd.get_dummies(X, columns=['job', 'marital', 'education', 'contact', 'poutcome'], drop_first=True).astype(int)


X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


<ipython-input-2-f7c94408d8ad>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['housing'] = X['housing'].map({'yes': 1, 'no': 0})
<ipython-input-2-f7c94408d8ad>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['loan'] = X['loan'].map({'yes': 1, 'no': 0})


Accuracy: 0.86


In [3]:
import pickle
with open('decision_tree_model.pkl', 'wb') as file:
    pickle.dump(clf, file)


In [4]:
import pickle
with open('feature_names.pkl', 'wb') as file:
    pickle.dump(X_train.columns.tolist(), file)

In [5]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.8 MB/s eta 0:00:00


In [6]:
import pandas as pd
import gradio as gr
from sklearn.tree import DecisionTreeClassifier
import pickle

In [7]:
with open('decision_tree_model.pkl', 'rb') as file:
    model = pickle.load(file)


with open('feature_names.pkl', 'rb') as file:
    feature_names = pickle.load(file)

def preprocess_input(age, job, marital, education, balance, housing, loan, contact, duration, campaign, poutcome):
    input_data = {
        'age': [age],
        'job': [job],
        'marital': [marital],
        'education': [education],
        'balance': [balance],
        'housing': [1 if housing == 'yes' else 0],
        'loan': [1 if loan == 'yes' else 0],
        'contact': [contact],
        'duration': [duration],
        'campaign': [campaign],
        'poutcome': [poutcome]
    }

    df = pd.DataFrame(input_data)

    df_encoded = pd.get_dummies(df, columns=['job', 'marital', 'education', 'contact', 'poutcome'])

    for col in feature_names:
        if col not in df_encoded.columns:
            df_encoded[col] = 0

    df_encoded = df_encoded[feature_names]

    return df_encoded

def predict_purchase(age, job, marital, education, balance, housing, loan, contact, duration, campaign, poutcome):
    try:
        print(f"Input received: {age}, {job}, {marital}, {education}, {balance}, {housing}, {loan}, {contact}, {duration}, {campaign}, {poutcome}")

        X_input = preprocess_input(age, job, marital, education, balance, housing, loan, contact, duration, campaign, poutcome)
        print(f"Preprocessed input shape: {X_input.shape}")
        print(f"Preprocessed input columns: {X_input.columns}")

        prediction = model.predict(X_input)[0]
        confidence = model.predict_proba(X_input)[0][prediction]

        result = "Yes" if prediction == 1 else "No"
        confidence_str = f"{confidence:.2%}"

        print(f"Prediction: {result}, Confidence: {confidence_str}")

        return result, confidence_str
    except Exception as e:
        error_msg = f"An error occurred: {str(e)}\n{traceback.format_exc()}"
        print(error_msg)
        return "Error", "Error"

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🛍️ Customer Purchase Prediction")
    gr.Markdown("Predict whether a customer will make a purchase based on their demographic and behavioral data.")

    with gr.Row():
        with gr.Column():
            age = gr.Slider(18, 100, label="Age", step=1)
            job = gr.Dropdown(choices=['management', 'technician', 'entrepreneur', 'blue-collar', 'unknown', 'retired', 'admin.', 'services', 'self-employed', 'unemployed', 'housemaid', 'student'], label="Job")
            marital = gr.Radio(['married', 'single', 'divorced'], label="Marital Status")
            education = gr.Dropdown(choices=['primary', 'secondary', 'tertiary', 'unknown'], label="Education")
            balance = gr.Number(label="Balance")

        with gr.Column():
            housing = gr.Radio(['yes', 'no'], label="Housing Loan")
            loan = gr.Radio(['yes', 'no'], label="Personal Loan")
            contact = gr.Dropdown(choices=['unknown', 'cellular', 'telephone'], label="Contact Method")
            duration = gr.Slider(0, 5000, label="Last Contact Duration (seconds)", step=1)
            campaign = gr.Slider(1, 50, label="Number of Contacts", step=1)
            poutcome = gr.Dropdown(choices=['unknown', 'failure', 'other', 'success'], label="Previous Campaign Outcome")

    submit_btn = gr.Button("Predict Purchase", variant="primary")

    with gr.Row():
        prediction_output = gr.Label(label="Prediction")
        confidence_output = gr.Label(label="Confidence")

    submit_btn.click(
        fn=predict_purchase,
        inputs=[age, job, marital, education, balance, housing, loan, contact, duration, campaign, poutcome],
        outputs=[prediction_output, confidence_output]
    )

    gr.Markdown("## How to use")
    gr.Markdown("1. Adjust the sliders and select options to input customer data.")
    gr.Markdown("2. Click the 'Predict Purchase' button to see the prediction.")
    gr.Markdown("3. The model will predict whether the customer is likely to make a purchase and show the confidence level.")

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://498e7456c78c743778.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
